# Required Libraries and data

In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout



# Load the dataset
data = pd.read_excel("outpute.xlsx")

# Tokenize the text
data["Tokenized Text"] = data["Cleaned Text"].apply(word_tokenize)

# Save the preprocessed dataset
data.to_excel("preprocessed_output.xlsx", index=False)

data.head


<bound method NDFrame.head of             text_id                                               text task_1  \
0     hasoc_hi_5556  बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...    NOT   
1     hasoc_hi_5648  सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...    HOF   
2      hasoc_hi_164  तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...    HOF   
3     hasoc_hi_3530  बीजेपी  आकाश विजयवर्गीय जेल से रिहा, जमानत मिल...    NOT   
4     hasoc_hi_5206  चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...    NOT   
...             ...                                                ...    ...   
4660  hasoc_hi_6606  पाकिस्तान ने हिंदुओं के ख़िलाफ़ बोलने वाले को ...    NOT   
4661  hasoc_hi_4931  कोहली है #नेहरू नहीं जो अंग्रेजों के तलवे चाटन...    HOF   
4662  hasoc_hi_1059       परशुराम? वही जिसने अपनी मां की हत्या की थीं?    NOT   
4663  hasoc_hi_5429  जिस देश में #कन्हैया_कुमार जैसा पढ़ा लिखा युवा...    HOF   
4664  hasoc_hi_1656  इनके बापों मैं भी दम नहीं जो भारत को इस्लामिक ...    HOF  

# CNN MODEL

In [ ]:
data = pd.read_excel("preprocessed_output.xlsx")                             # Load the preprocessed dataset

tokenizer = Tokenizer()                                                      # Convert lists of strings to sequences of integers
tokenizer.fit_on_texts(data["Tokenized Text"])
sequences = tokenizer.texts_to_sequences(data["Tokenized Text"])

max_sequence_length = 100                                                    # Pad sequences to a fixed length
X = pad_sequences(sequences, maxlen=max_sequence_length)

y = data["task_1"].apply(lambda x: 1 if x == "HOF" else 0)                  # Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

embedding_dim = 100                                                        # Define the CNN model

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))  # Train the model
 
y_pred = model.predict(X_test)                                                           # Make predictions
y_pred = np.round(y_pred).flatten()

report = classification_report(y_test, y_pred)                                          # Calculate evaluation metrics
print(report)

# LSTM MODEL

In [ ]:
data = pd.read_excel("preprocessed_output.xlsx")                                     # Load the preprocessed dataset

tokenizer = Tokenizer()                                                     # Convert lists of strings to sequences of integers
tokenizer.fit_on_texts(data["Tokenized Text"])
sequences = tokenizer.texts_to_sequences(data["Tokenized Text"])

max_sequence_length = 100                                                  # Pad sequences to a fixed length
X = pad_sequences(sequences, maxlen=max_sequence_length)

y = data["task_1"].apply(lambda x: 1 if x == "HOF" else 0)                # Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

embedding_dim = 100                                                      # Define the LSTM model

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))     # Train the model

y_pred = model.predict(X_test)                                                             # Make predictions
y_pred = np.round(y_pred).flatten()

report = classification_report(y_test, y_pred)                                            # Calculate evaluation metrics
print(report)


# BI-LSTM MODEL

In [ ]:
data = pd.read_excel("preprocessed_output.xlsx")                                    # Load the preprocessed dataset

tokenizer = Tokenizer()                                                     # Convert lists of strings to sequences of integers
tokenizer.fit_on_texts(data["Tokenized Text"])
sequences = tokenizer.texts_to_sequences(data["Tokenized Text"])

max_sequence_length = 100                                                  # Pad sequences to a fixed length
X = pad_sequences(sequences, maxlen=max_sequence_length)

y = data["task_1"].apply(lambda x: 1 if x == "HOF" else 0)      # Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

embedding_dim = 100                                            # Define the BiLSTM model

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))    # Train the model

y_pred = model.predict(X_test)                                                            # Make predictions
y_pred = np.round(y_pred).flatten()

report = classification_report(y_test, y_pred)                                           # Calculate evaluation metrics
print(report)